# Introduction: objective

Lets assume you never visit US before. And you want to eat only burger. So you are finding a place where you can find a lot of number of shops of burger.The problem we aim to solve is to analyze the burger stores’ locations in the major US cities. Find the best place for our tourist for good burger.



# Data:
I will use the FourSquare API to collect data about locations of burger stores in 5 major US cities which are: New York,NY, San Francisco, CA,Jersey City, NJ, Boston, MA and Chicago,IL.These are one of the most populated US cities and I am where tourist can find good restaurant for burger.

1) Geographical coordinate of cities 

2) Favourite restaurants 

3) Shopping complex and facilities for families.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'UP1DLMNZTHCTJVGIJCZYTRNO3BHYKOGZ3GPVKRMN5KJSX5BS' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: UP1DLMNZTHCTJVGIJCZYTRNO3BHYKOGZ3GPVKRMN5KJSX5BS


In [3]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") 
    results[city] = requests.get(url).json()


In [ ]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [ ]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

In [ ]:
maps[cities[0]]

In [ ]:
maps[cities[1]]

In [ ]:
maps[cities[2]]

In [ ]:
maps[cities[3]]

In [ ]:
maps[cities[4]]

We can see that New York and Jersey City are the most dense cities with burger places. And better than that, they are just one shore away.

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the burger places which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates

In [ ]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02155584712137211
Chicago, IL
Mean Distance from Mean coordinates
0.052804625789718715
San Francisco, CA
Mean Distance from Mean coordinates
0.028633234082617753
Jersey City, NJ
Mean Distance from Mean coordinates
0.02995019447478735
Boston, MA
Mean Distance from Mean coordinates
0.03512614671610634

In [ ]:
maps[cities[0]]

In [ ]:
maps[cities[1]]

In [ ]:
maps[cities[2]]

In [ ]:
maps[cities[3]]

In [ ]:
maps[cities[4]]


Another observation is that there is one really far away burger store which would possible increase its score to be beaten by New York So let's try to remove it and calculate it again

In [ ]:
city = 'Jersey City, NJ'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

Jersey City, NJ
Mean Distance from Mean coordinates
0.021995384838861428
That puts Jersey City back in the first place which makes our tourist happy.